In [4]:
from utils import *

from nltk.corpus import stopwords

# 1. Preprocessing Data

In [2]:
# preprocessing training data
train = processing_documents('training/')
train.head()

,class,document_content
0,gold,carolin mines has new assay results vancouver...
1,gold,starrex links share price to assay speculatio...
2,gold,correctedhecla to buy mine stake from bp coeu...
3,gold,black miners support s african mines takeover...
4,gold,chinas heilongjiang province boosts gold outp...


In [3]:
# preprocessing training data
test = processing_documents('test/')
test.head()

,class,document_content
0,earn,willamette industries th qtr net portland ore...
1,earn,amatil proposes twoforfive bonus share issue ...
2,earn,bowater pretax profits rise mln stg london ap...
3,earn,bowater industries profit exceed expectations...
4,earn,citibank norway unit loses six mln crowns in ...


In [ ]:
# english stopwords
en_stopwords = set(stopwords.words('english'))